In [1]:
import pandas as pd
import numpy as np
import datetime
from functools import reduce

data = pd.read_csv("../Resources/sars_cases.csv")
death = pd.read_csv("../Resources/sars_deaths.csv")
hospitalization = pd.read_csv("../Resources/sars_hospitalizations.csv")
population = pd.read_csv("population.csv")

#adding suffix
hospitalization = hospitalization.add_suffix("_hospitalization")
#census = pd.read_csv("../Resources/Census_Wash_Counties_Income_Poverty.csv")

cleandata = pd.DataFrame.copy(data)

day = pd.Series(cleandata["WeekStartDate"]).str.split("/").str[1]
month = pd.Series(cleandata["WeekStartDate"]).str.split("/").str[0]
year = pd.Series(cleandata["WeekStartDate"]).str.split("/").str[2]

month = month.map(lambda x: x.zfill(2))
day = day.map(lambda x: x.zfill(2))


cleandata["WeekStartDate"] = year + "-" + month + "-" + day




dataframes = [cleandata, death, hospitalization]

#mergedf = reduce(lambda  left,right: pd.merge(left,right,on=['County', "WeekStartDate"], suffixes= ("_cases", "_death", "_hospitalization"), how='outer'), dataframes)

mergedf = pd.merge(cleandata, death, on = ["County", "WeekStartDate"], suffixes = ("_cases", "_death"), how= "outer", indicator = True)#.fillna(0)
#mergedf[mergedf["_merge"] == "both"]

merge1 = pd.DataFrame(mergedf)

mergedf2 = pd.merge(merge1, hospitalization, left_on = ["County", "WeekStartDate"], right_on = ["County_hospitalization", "WeekStartDate_hospitalization"], how = "outer", indicator = "exists")

#mergedf.fillna(0)

mergedf2.to_csv("MergeWeek.csv")


In [2]:
mergedf3 = mergedf2
mergedf3["Month"] = mergedf3["WeekStartDate"].str.split("-").str[1]

mergemonth = mergedf3.groupby(["County", "Month"])
mergemonth = mergemonth[['NewPos_All', 'Age 0-19_cases',
       'Age 20-39_cases', 'Age 40-59_cases', 'Age 60-79_cases',
       'Age 80+_cases', 'Positive UnkAge_cases', 'dtm_updated_cases', 'Deaths',
       'Age 0-19_death', 'Age 20-39_death', 'Age 40-59_death',
       'Age 60-79_death', 'Age 80+_death', 'Positive UnkAge_death',
       'dtm_updated_death', 'County_hospitalization',
       'Hospitalizations_hospitalization',
       'Age 0-19_hospitalization', 'Age 20-39_hospitalization',
       'Age 40-59_hospitalization', 'Age 60-79_hospitalization',
       'Age 80+_hospitalization', 'Positive UnkAge_hospitalization',
       'dtm_updated_hospitalization']].sum()
mergemonth.to_csv("MergeMonth.csv")



In [3]:
census = pd.read_csv("../census_politics_merge_mike2.csv")

mergedf4 = mergemonth.reset_index()
mergedf5 = pd.merge(mergedf4, census, on = "County", how="outer")
mergedf5 = mergedf5.drop(columns=['Counter'])
#mergedf5.to_csv("Final Merge 7_15_20")


In [7]:
county_gb = mergedf4.groupby("County")
county_sum = county_gb.sum()
county_sum
merge_census = pd.merge(county_sum, census, on = "County", how="outer")
covid_reg = pd.merge(merge_census, population, on = "County", how="outer")
covid_reg = covid_reg.drop([31])
covid_reg = covid_reg.drop(columns=['Counter'])
covid_reg["Pop_per100k"] = covid_reg["Population"]/100000
covid_reg = covid_reg.fillna(0)
covid_reg["Case_per100k"] = covid_reg["NewPos_All"]/covid_reg["Pop_per100k"]
covid_reg["Deaths_per100k"] = covid_reg["Deaths"]/covid_reg["Pop_per100k"]
covid_reg["Hoptializations_per100k"] = covid_reg["Hospitalizations_hospitalization"]/covid_reg["Pop_per100k"]
covid_reg.to_csv("Final Merge 7_16_20.csv")

In [18]:
print(covid_reg["NewPos_All"].sum())
covid_reg["Pop_per100k"].sum()

covid_reg["Case_per100k"].sum()

#covid_reg

35753.0


13949.473031853855

In [17]:
covid_reg["Case_per100k"].sum()
covid_reg["Population"].sum()

7614875.0

In [22]:
covid_reg.head(20)

,County,NewPos_All,Age 0-19_cases,Age 20-39_cases,Age 40-59_cases,Age 60-79_cases,Age 80+_cases,Positive UnkAge_cases,Deaths,Age 0-19_death,...,Positive UnkAge_hospitalization,Political_Party,Percent Below Poverty Level,Household Income Median,Household Income Mean,Population,Pop_per100k,Case_per100k,Deaths_per100k,Hoptializations_per100k
0,Adams County,166.0,17.0,63.0,59.0,25.0,0.0,2.0,0.0,0.0,...,0.0,Republican,25.2,49142.0,61370.0,19983.0,0.19983,830.706100,0.000000,45.038283
1,Asotin County,21.0,1.0,1.0,10.0,7.0,2.0,0.0,2.0,0.0,...,0.0,Republican,13.0,50423.0,65079.0,22582.0,0.22582,92.994420,8.856611,4.428306
2,Benton County,1956.0,185.0,747.0,572.0,285.0,166.0,1.0,85.0,0.0,...,0.0,Republican,12.8,65650.0,83941.0,204390.0,2.04390,956.993982,41.587162,114.976271
3,Chelan County,403.0,47.0,175.0,127.0,42.0,12.0,0.0,6.0,0.0,...,0.0,Republican,11.6,56135.0,75117.0,77200.0,0.77200,522.020725,7.772021,23.316062
4,Clallam County,43.0,6.0,15.0,10.0,10.0,2.0,0.0,0.0,0.0,...,0.0,Democratic,15.9,49913.0,65858.0,77331.0,0.77331,55.605126,0.000000,3.879427
5,Clark County,922.0,94.0,335.0,304.0,145.0,44.0,0.0,30.0,0.0,...,0.0,Democratic,9.3,71636.0,91321.0,488241.0,4.88241,188.841167,6.144507,22.734674
6,Columbia County,8.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,Republican,11.6,51111.0,70516.0,3985.0,0.03985,200.752823,0.000000,0.000000
7,Cowlitz County,208.0,24.0,83.0,57.0,40.0,4.0,0.0,0.0,0.0,...,0.0,Republican,16.0,51752.0,67765.0,110593.0,1.10593,188.077003,0.000000,18.084327
8,Douglas County,279.0,40.0,120.0,88.0,22.0,8.0,1.0,3.0,0.0,...,1.0,Republican,13.3,60452.0,77297.0,43429.0,0.43429,642.427871,6.907827,29.933915
9,Franklin County,1898.0,200.0,727.0,700.0,229.0,39.0,3.0,33.0,0.0,...,0.0,Republican,15.7,62002.0,76571.0,95222.0,0.95222,1993.236857,34.655857,170.128752
